In [1]:
import json
import os
import re
import numpy as np
import pickle
from collections import Counter
from preprocessing import CharacterIndexer, SlotIndexer, IntentIndexer
from gensim.models import Word2Vec

/home/derek/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/derek/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### read data

the `PlayMusic` file has a bad entry so the commented cell must be run once, then the original `PlayMusic` file deleted.

In [2]:
# # fix bad line in the PlayMusic file
# # then manually rename
# def decode(s, encoding="utf8", errors="ignore"):
#     return s.decode(encoding=encoding, errors=errors)

# raw_json_file = open('data/snips/train/train_PlayMusic_full.json', 'rb')
# raw_json = decode(bytes(raw_json_file.read()))
# all_str = raw_json[ raw_json.find("{"): ]
# all_obj = json.loads(all_str)
# with open('data/snips/train/train_PlayMusic_full_fixed.json', 'w') as outfile:
#     json.dump(all_obj, outfile)

In [3]:
train_sents, train_tags, train_intents = [], [], []
path = 'data/snips/train'
for filename in os.listdir(path):
    if 'json' in filename:
        with open(path + '/' + filename, encoding='utf8') as json_file:
            intent = filename.split('_')[1]
            print(filename, intent)
    #         try:
            data = json.load(json_file)
            data = data[intent]
            for sent in data:
                s, t = [], []
                for dct in sent['data']:
                    if 'entity' in dct.keys():
                        t.append(dct['entity'])
                        s.append(dct['text'])
                    else:
                        t.append("NONE")
                        s.append(dct['text'])
                train_sents.append(s)
                train_tags.append(t)
                train_intents.append(intent)
#         except UnicodeDecodeError:
#             pass

train_PlayMusic_full_fixed.json PlayMusic
train_SearchCreativeWork_full.json SearchCreativeWork
train_BookRestaurant_full.json BookRestaurant
train_RateBook_full.json RateBook
train_GetWeather_full.json GetWeather
train_AddToPlaylist_full.json AddToPlaylist
train_SearchScreeningEvent_full.json SearchScreeningEvent


In [4]:
len(train_sents), len(train_tags), len(train_intents)

(13784, 13784, 13784)

In [5]:
Counter(train_intents).most_common()

[('PlayMusic', 2000),
 ('GetWeather', 2000),
 ('BookRestaurant', 1973),
 ('SearchScreeningEvent', 1959),
 ('RateBook', 1956),
 ('SearchCreativeWork', 1954),
 ('AddToPlaylist', 1942)]

In [6]:
val_sents, val_tags, val_intents = [], [], []
path = 'data/snips/validate'
for filename in os.listdir(path):
    if 'json' in filename:
        with open(path + '/' + filename) as json_file:
            intent = filename.split('_')[1]
            intent = intent.split('.')[0]
            print(filename, intent)
#             try:
            data = json.load(json_file)
            data = data[intent]
            for sent in data:
                s, t = [], []
                for dct in sent['data']:
                    if 'entity' in dct.keys():
                        t.append(dct['entity'])
                        s.append(dct['text'])
                    else:
                        t.append("NONE")
                        s.append(dct['text'])
                val_sents.append(s)
                val_tags.append(t)
                val_intents.append(intent)
#             except UnicodeDecodeError:
#                 pass

validate_SearchCreativeWork.json SearchCreativeWork
validate_AddToPlaylist.json AddToPlaylist
validate_BookRestaurant.json BookRestaurant
validate_SearchScreeningEvent.json SearchScreeningEvent
validate_PlayMusic.json PlayMusic
validate_GetWeather.json GetWeather
validate_RateBook.json RateBook


In [7]:
len(val_sents), len(val_tags), len(val_intents)

(700, 700, 700)

In [8]:
Counter(val_intents).most_common()

[('SearchCreativeWork', 100),
 ('AddToPlaylist', 100),
 ('BookRestaurant', 100),
 ('SearchScreeningEvent', 100),
 ('PlayMusic', 100),
 ('GetWeather', 100),
 ('RateBook', 100)]

In [9]:
print(train_sents[0])
print(train_tags[0])
print(train_intents[0])

['I need to hear the ', 'song', ' ', 'Aspro Mavro', ' from ', 'Bill Szymczyk', ' on ', 'Youtube']
['NONE', 'music_item', 'NONE', 'track', 'NONE', 'artist', 'NONE', 'service']
PlayMusic


In [10]:
# preprocess sentences
def cleanup(sentlist, taglist):
    newsents = []
    newtags  = []
    for idx, sent in enumerate(sentlist):
        newsent, newtag = [], []
        for jdx, phrase in  enumerate(sent):
            for c in ['.', ',', '!', '?', ]:
                phrase = phrase.replace(c, '')
            tt = phrase.split()
            for kdx, t in enumerate(tt):
                # digit replacement
                # if t.isdigit():
                #     newsent.append(digits.get(t, '##'))
                # else:
                newsent.append(t.lower())
                newtag.append(taglist[idx][jdx])
        newsents.append(newsent)
        newtags.append(newtag)
    return newsents, newtags

In [11]:
train_sents, train_tags = cleanup(train_sents, train_tags)

In [12]:
val_sents, val_tags = cleanup(val_sents, val_tags)

In [13]:
print(train_sents[0])
print(train_tags[0])
print(train_intents[0])

['i', 'need', 'to', 'hear', 'the', 'song', 'aspro', 'mavro', 'from', 'bill', 'szymczyk', 'on', 'youtube']
['NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'music_item', 'track', 'track', 'NONE', 'artist', 'artist', 'NONE', 'service']
PlayMusic


In [35]:
len(list(set([t for s in train_tags for t in s])))

40

In [34]:
Counter(list([t for s in train_tags for t in s])).most_common(10)

[('NONE', 59611),
 ('object_name', 10792),
 ('playlist', 5511),
 ('timeRange', 4787),
 ('object_type', 4216),
 ('artist', 3793),
 ('movie_name', 2794),
 ('spatial_relation', 2103),
 ('rating_value', 1957),
 ('city', 1829)]

### train word2vec embeddings

using pretrained embeddings has been found to increase performance in various NLP tasks.

we could train on an external corpus such as brown but we will just use the training data

In [14]:
# train and save model
model = Word2Vec(train_sents, size=200, min_count=1, window=3, workers=3, iter=5)
model.save('model/snips_w2v.gensimmodel')
print('training done!')

training done!


In [15]:
# get model vocabulary
vocab = dict([(k, v.index) for k, v in model.wv.vocab.items()])

In [16]:
# test
model.wv.most_similar('hear')

[('listen', 0.997948169708252),
 ('would', 0.9847648739814758),
 ('want', 0.9830597043037415),
 ('w', 0.9820548295974731),
 ('watch', 0.9763119220733643),
 ('to', 0.9758093357086182),
 ('insult', 0.972419798374176),
 ('go', 0.9699915647506714),
 ('see', 0.9680747389793396),
 ('kill', 0.9657482504844666)]

### encode data

In [17]:
vocab = list(set([w for s in train_sents for w in s]))
len(vocab)

11895

In [18]:
VOCABSIZE = 10000

In [19]:
lens = [len(s) for s in train_sents]
_MAXLEN = int(np.round(np.mean(lens) + 2*np.std(lens)))
_MAXLEN

15

In [20]:
sum([1 if lens[i] <= _MAXLEN else 0 for i in range(len(lens))])/len(lens)

0.9640887986070806

In [21]:
sent_indexer = CharacterIndexer(max_sent_len=15, max_word_mode='std', max_word_vocab=10000)

In [22]:
sent_indexer.fit(train_sents, verbose=True)

fit(): splitting...
fit(): max word len set to 9
fit(): creating conversion dictionaries...
fit(): tru word vocab: 10000
fit(): tru char vocab: 91
fit(): done!


In [23]:
# transform the sentence data
trn_text_idx, trn_char_idx = sent_indexer.transform(train_sents)
tst_text_idx, tst_char_idx = sent_indexer.transform(val_sents)
trn_text_idx.shape, trn_char_idx.shape, tst_text_idx.shape, tst_char_idx.shape

((13784, 15), (13784, 15, 9), (700, 15), (700, 15, 9))

In [24]:
# transform the slot data
slot_indexer = SlotIndexer(max_len=15)
slot_indexer.fit(train_tags)
trn_slot_idx = slot_indexer.transform(train_tags)
tst_slot_idx = slot_indexer.transform(val_tags)
trn_slot_idx.shape, tst_slot_idx.shape

((13784, 15, 1), (700, 15, 1))

In [25]:
# transform the intent data
int_indexer = IntentIndexer()
int_indexer.fit(train_intents)
trn_int_idx = int_indexer.transform(train_intents)
tst_int_idx = int_indexer.transform(val_intents)
trn_int_idx.shape, tst_int_idx.shape

((13784, 1), (700, 1))

### save data

In [26]:
pickle.dump(train_sents,   open('data/snips/train_sents.pkl', 'wb'))
pickle.dump(train_tags,    open('data/snips/train_slots.pkl', 'wb'))
pickle.dump(train_intents, open('data/snips/train_intents.pkl', 'wb'))
pickle.dump(val_sents,   open('data/snips/test_sents.pkl', 'wb'))
pickle.dump(val_tags,    open('data/snips/test_slots.pkl', 'wb'))
pickle.dump(val_intents, open('data/snips/test_intents.pkl', 'wb'))

In [27]:
np.save('encoded/snips_x_trn_text.npy', trn_text_idx)
np.save('encoded/snips_x_tst_text.npy', tst_text_idx)

np.save('encoded/snips_x_trn_char.npy', trn_char_idx)
np.save('encoded/snips_x_tst_char.npy', tst_char_idx)

np.save('encoded/snips_y_trn_slot.npy', trn_slot_idx)
np.save('encoded/snips_y_tst_slot.npy', tst_slot_idx)

np.save('encoded/snips_y_trn_ints.npy', trn_int_idx)
np.save('encoded/snips_y_tst_ints.npy', tst_int_idx)

In [28]:
pickle.dump(sent_indexer, open("encoded/snips_sent_indexer.pkl", "wb"))
pickle.dump(slot_indexer, open("encoded/snips_slot_indexer.pkl", "wb"))
pickle.dump(int_indexer,  open("encoded/snips_int_indexer.pkl", "wb"))

In [29]:
pickle.dump(vocab,  open("model/snips_w2v_vocab.pkl", "wb"))